<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd

movie = pd.read_excel('/content/movie2.xlsx')

In [ ]:
movie.columns

In [69]:
## 결측치 처리1

movie2 = movie.drop(movie.columns[movie.isna().sum() >= 1000].values, axis=1)

In [70]:
## 오기입 데이터 처리
idx = movie2[['10대_비율', '20대_비율', '30대_비율', '40대_비율',
       '50대_비율']].sum(axis=1) != 0

movie3 = movie2.loc[idx]

In [ ]:
movie3.info()
movie3.columns

In [71]:
## 결측치 처리2
## OST ~ 연출 => drop
movie4 = movie3.dropna()


In [ ]:
## OST ~ 연출 => impute
list1 = ['OST', '영상미', '스토리', '연기', '연출']

for col in list1 :
  col_mean = movie3[col].loc[movie3[col].notna()].mean()
  movie3[col].fillna(col_mean, inplace=True)


In [73]:
movie5 = movie3.dropna()

In [ ]:
movie4 # 결측치 다 드롭
movie5 # impute

In [ ]:
movie4.info()

In [95]:
# 관객수 처리
movie4_tmp = movie4['관객수_x'].str.replace('명', '').str.replace(',', '')

movie4_tmp2 = movie4_tmp.loc[movie4_tmp.str.contains('만')].str.replace('만', '').astype('float') * 10000

movie4_tmp2.index

movie4_tmp.loc[movie4_tmp2.index] = movie4_tmp2



In [ ]:
movie4['관객수'] = movie4_tmp.astype('float')

In [102]:
movie4_2 = movie4.drop('관객수_x', axis=1)

In [104]:
target = movie4_2['관객수']

In [ ]:
## 데이터 전처리
### 범주 -> 인코딩

### 제목 -> 제외

### 개봉 -> 방학시즌,
### 개봉 시즌 -> 계절
### 등급 -> 원핫
### 장르 -> 한 영화에 장르가 중첩 -> 처리해서 사용
### 국가 -> 사용
### 러닝타임 -> 수치로 바꿔서 사용
### 배급사 -> 빈도수 많은 거 위주로 사용
### 참여수 -> 제외. 나중에 고려
### 감독 -> 타겟 인코딩, 레이블 인코딩

movie4_2['감독'].value_counts()

### 관객수와 참여수 비례

In [151]:
### 개봉 시즌 -> 4계절로 바꾸기

def convert_season(month) :
  if month in [12,1,2] :
    return '겨울'
  elif month in [3,4,5] :
    return '봄'
  elif month in [6,7,8] :
    return '여름'
  else :
    return '가을'

movie4_2['개봉 시즌'] = movie4_2['개봉'].str[5:7].astype('int').map(convert_season)

array(['여름', '겨울', '봄', '가을'], dtype=object)

In [160]:
### 장르 -> 한 영화에 장르가 중첩 -> 처리해서 사용. 가장 앞에 있는 장르만 사용
movie4_2['대표 장르'] = movie4_2['장르'].str.split(',').str[0]

In [170]:
### 국가 -> 사용 -> 제일 앞에 있는 국가를 대표로 선택
movie4_2['대표 국가'] = movie4_2['국가'].str.split(',').str[0]

In [174]:
### 러닝타임 -> 수치로 바꿔서 사용 -> 분 제거하고 astype

movie4_2['러닝타임(수치)'] = movie4_2['러닝타임'].str.replace('분', '').astype('int')

In [195]:
### 배급사 -> 빈도수 많은 거 위주로 사용 -> 국가와 비슷하게 해결

values = movie4_2['배급'].value_counts()

x = list(range(values.size))

idx = values >= 20

use_enter = values[idx].index.values

def enter_20_times(enter):
  if enter in use_enter :
    return enter
  return '기타'

movie4_2['주요 배급사'] = movie4_2['배급'].map(enter_20_times)

# df['host_name'] = df['host_name'].map(hname_5_times)


# range(0, movie4_2['배급'].value_counts())

# movie4_2['배급'].value_counts() >= 20

In [199]:
### 참여수 -> 제외. 나중에 고려 -> drop
movie4_3 = movie4_2.drop('참여수', axis=1)

In [200]:
### 감독 -> 타겟 인코딩, 레이블 인코딩 -> 타겟인코딩
movie4_3["감독_인코딩"] = movie4_3.groupby("감독")["관객수"].transform("mean")


In [ ]:
  # 과학적 표기법 대신 소수점 5자리까지 나타낸다.
pd.options.display.float_format = '{:.5f}'.format
movie4_3["감독_인코딩"]

In [ ]:
movie4_3.info()

# 제목, 개봉, 장르, 국가, 러닝타임, 배급, 감독

movie4_4 = movie4_3.drop(['제목', '개봉', '장르', '국가', '러닝타임', '배급', '감독'], axis=1)

In [ ]:
## 원핫 인코딩

cate = movie4_4.select_dtypes('object')
target = movie4_4['관객수']
num = movie4_4.drop(cate.columns, axis=1).drop('관객수', axis=1)

num.columns
cate.columns
target



In [226]:
cate_1hot = pd.get_dummies(cate)

In [230]:
train = pd.concat([num, cate_1hot], axis=1)

In [233]:
from sklearn.model_selection import train_test_split

trd, tsd, trt, tst = train_test_split(train, target, train_size=0.8, random_state=42)

In [252]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=100)


ridge.fit(trd, trt);
ridge.score(trd, trt), ridge.score(tsd, tst)


(0.8258002720775217, 0.7785527019416066)

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(ridge, tsd, tst, cv=5).mean()
cross_val_score(ridge, trd, trt, cv=5).mean()